In [50]:
import roboticstoolbox as rtb
import swift
import numpy as np
import spatialmath as sm
from spatialmath import SE3
import spatialgeometry as sg # to visualise the basic shapes and meshes of the geometry

In [51]:
# Creating a swift instance
env = swift.Swift()
env.launch(realtime=True) #realtime = Ture helps the Swift to run as close to real time as it can otherwise it will 
                          #perform as quickly as possible.

In [52]:
# initialising the robot
IIWA = rtb.models.LBR()
IIWA.q = IIWA.qr # Initializing the joint coordinates 'q' with a predifened set of coordinates 'qr' (qr is the robot home position)

IIWA.qd= [1,0,0,0,0,0,0.5] # initializing the joint velocities 'qd' in rad/s; the last one is the end effector.

print(IIWA) # just printing the coordinates

#Add panda to swift
env.add(IIWA, robot_alpha=1, collision_alpha=0) # 'robot_alpha' keayword is for robot opacity, it is now set to 100%
                                                    # 'collision_alpha' is for collision opacity (with cylinders, spheres, etc)
                                                    # now it is set to 0%

ERobot: kuka_lbr_iiwa_14_r820 (by Kuka), 7 joints (RRRRRRR), 2 branches, geometry, collision
┌─────┬───────────┬───────┬───────────┬───────────────────────────────────┐
│link │   link    │ joint │  parent   │        ETS: parent to link        │
├─────┼───────────┼───────┼───────────┼───────────────────────────────────┤
│   0 │ base_link │       │ BASE      │ SE3()                             │
│   1 │ link_1    │     0 │ base_link │ SE3() ⊕ Rz(q0)                    │
│   2 │ link_2    │     1 │ link_1    │ SE3(-0.0004362, 0, 0.36) ⊕ Ry(q1) │
│   3 │ link_3    │     2 │ link_2    │ SE3() ⊕ Rz(q2)                    │
│   4 │ link_4    │     3 │ link_3    │ SE3(0.0004362, 0, 0.42) ⊕ Ry(-q3) │
│   5 │ link_5    │     4 │ link_4    │ SE3() ⊕ Rz(q4)                    │
│   6 │ link_6    │     5 │ link_5    │ SE3(0, 0, 0.4) ⊕ Ry(q5)           │
│   7 │ link_7    │     6 │ link_6    │ SE3() ⊕ Rz(q6)                    │
│   8 │ @tool0    │       │ link_7    │ SE3(0, 0, 0.126)               

0

In [53]:
# stepping the environment for swift to perform the motion
for _ in range(100):      # '_' means the loop variable isn't important here, we just want it to iterate 100 times.
    env.step(0.05)        # control the step counts for smooth flow.

In [54]:
# Setting the end effector position
EndPoint= IIWA.fkine(IIWA.q)*sm.SE3.Tx(0.3)*sm.SE3.Ty(0.4)*sm.SE3.Tz(0.4-0.2)*sm.SE3.Rx(np.pi/3)
# With this we are 1st finding the positon of the end effector (using forward kinematics) and then offseting this position by using the translation operator 'Tx/y/z' and 
# rotational operator 'Rx/y/z'. The dimentions are in meters and radians
# we can manipulate this to take the end effector whereever we want and here we need to set a virtual offset at the end using translation or rotation.

GoalPoint = IIWA.fkine(IIWA.q)*sm.SE3.Tx(0.3)*sm.SE3.Ty(0.4)*sm.SE3.Tz(0.4)
# Defining a point for the axis (For my project I can use it as my desired point/goal point that is somewhere in top surface of the abdomen)

axes = sg.Axes(length=0.1, base=GoalPoint) # Putting some axis in the swift environment, here it is a right handed coordinates 
                                     # with lenght= 0.1 and pos= GoalPoint
                                     # This is a axis created with some random points
env.add(axes)

multiple end-effectors present, ambiguous, using self.ee_links[0]


/Users/sujaymukherjee/anaconda3/envs/Internship/lib/python3.10/site-packages/spatialgeometry/geom/Shape.py:81: FutureWarning: base kwarg is deprecated, use pose instead
  warn("base kwarg is deprecated, use pose instead", FutureWarning)


1

In [55]:
# Printing the EndPoint and the GoalPoint to visualise
print(EndPoint)
print(GoalPoint)

  -0.9953    0.008171 -0.09605  -0.5485    
   0.08704   0.5044   -0.8591    0.4749    
   0.04142  -0.8635   -0.5027    0.4379    
   0         0         0         1         

  -0.9953    0.08726  -0.04095  -0.5567    
   0.08704   0.9962    0.007231  0.4763    
   0.04142   0.003633 -0.9991    0.238     
   0         0         0         1         



In [56]:
# Making the controller aware of where it is now and where it has to move to

# Arrived at destination flag
arrived =  False

# A time step
dt = 0.01

# This loop is making the arm move from one point to another.
while not arrived:
    # v is a 6 vector representing the spatial error
    v, arrived = rtb.p_servo(IIWA.fkine(IIWA.q), EndPoint, gain=2, threshold=0.01)
      # keyword p_servo returns the error b/w one pose and another, here we are comparing b/w pos of end effector and
      # the desired pos 'EndPoint', 'gain' will control the movement speed of the arm from one pos to other, and 'threshold'
      # returns the sum of the error, if they come to the mentioned value 'arrived' will set to be true.

    J = IIWA.jacobe(IIWA.q) # will give the jacobian of the end effector.

    IIWA.qd = np.linalg.pinv(J) @ v  # desired joint velocity of the robot.

    #stepping the environment
    env.step(dt)

connection handler failed
Traceback (most recent call last):
  File "/Users/sujaymukherjee/anaconda3/envs/Internship/lib/python3.10/site-packages/websockets/legacy/server.py", line 236, in handler
    await self.ws_handler(self)
  File "/Users/sujaymukherjee/anaconda3/envs/Internship/lib/python3.10/site-packages/websockets/legacy/server.py", line 1175, in _ws_handler
    return await cast(
  File "/Users/sujaymukherjee/anaconda3/envs/Internship/lib/python3.10/site-packages/swift/SwiftRoute.py", line 320, in serve
    await self.expect_message(websocket, expected)
  File "/Users/sujaymukherjee/anaconda3/envs/Internship/lib/python3.10/site-packages/swift/SwiftRoute.py", line 325, in expect_message
    recieved = await websocket.recv()
  File "/Users/sujaymukherjee/anaconda3/envs/Internship/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 568, in recv
    await self.ensure_open()
  File "/Users/sujaymukherjee/anaconda3/envs/Internship/lib/python3.10/site-packages/websocke

KeyboardInterrupt: 

In [ ]:
# To visualize the robot (using plot refrains doing every step manully)
#### IIWA.plot(IIWA.qr,block=True) ######
 # this 'block' keyword argument stops the program from exiting, thus it will keep running untill manually interrupted. Else Swift would close.

#Stop the swift browser from closing
env.hold()

KeyboardInterrupt: 